<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/sharpness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title ##**Install** { display-mode: "form" }
%%capture
%cd /content
!pip install Pillow
from google.colab import files
import os
import os
import shutil
from google.colab import files
from PIL import Image
import numpy as np


In [ ]:
#@title ##**Upload images** { display-mode: "form" }

upload_folder = "/content/upload"
result_folder = "/content/results"

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder)

if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

basepath = os.getcwd()
uploaded = files.upload()
for filename in uploaded.keys():
    shutil.move(os.path.join(basepath, filename), os.path.join(upload_folder, filename))

In [10]:
#@title ##**Run** { display-mode: "form" }

if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

upload_folder = "/content/upload"
result_folder = "/content/results"

upload_files = set()
sharpness_factor = 3  # @param {type:"slider", min:0.0, max:7.0, step:0.1}
upscale_option = "height 720px"  # @param ["No", "height 720px", "height 1080px", "height 2160px", "2x", "4x"]

from PIL import ImageEnhance

filenames = os.listdir(upload_folder)
filenames.sort()

for filename in filenames:
    input_path = os.path.join(upload_folder, filename)
    output_path = os.path.join(result_folder, filename)

    original_image = Image.open(input_path)

    # Увеличиваем резкость с заданным коэффициентом
    enhancer = ImageEnhance.Sharpness(original_image)
    processed_image = enhancer.enhance(sharpness_factor)

    # Выполняем апскейл изображения, если выбран вариант отличный от "Нет"
    if upscale_option != "Нет":
        width, height = processed_image.size

        if upscale_option == "До высоты 720px":
            new_height = 720
            new_width = int(width * (new_height / height))
        elif upscale_option == "До высоты 1080px":
            new_height = 1080
            new_width = int(width * (new_height / height))
        elif upscale_option == "До высоты 2160px":
            new_height = 2160
            new_width = int(width * (new_height / height))
        elif upscale_option == "2x":
            new_width = width * 2
            new_height = height * 2
        elif upscale_option == "4x":
            new_width = width * 4
            new_height = height * 4

        # Выполняем изменение размера с наилучшим методом ресемплинга
        processed_image = processed_image.resize((new_width, new_height), Image.LANCZOS)

    # Сохраняем обработанное изображение
    processed_image.save(output_path)

In [ ]:
#@title ##**Visualize** { display-mode: "form" }

from IPython.display import display as ipy_display
import numpy as np
import PIL.Image

filenames_upload = os.listdir(upload_folder)
filenames_upload.sort()

filenames_output = os.listdir(result_folder)
filenames_output.sort()

for filename_input, filename_output in zip(filenames_upload, filenames_output):
    image_original = PIL.Image.open(os.path.join(upload_folder, filename_input))
    image_processed = PIL.Image.open(os.path.join(result_folder, filename_output))

    max_width = 500
    width_original, height_original = image_original.size
    width_processed, height_processed = image_processed.size
    if width_original > max_width:
        new_height = int(height_original * max_width / width_original)
        image_original = image_original.resize((max_width, new_height))
    if width_processed > max_width:
        new_height = int(height_processed * max_width / width_processed)
        image_processed = image_processed.resize((max_width, new_height))

    combined_image = PIL.Image.fromarray(np.hstack((np.array(image_original), np.array(image_processed))))
    ipy_display(combined_image)
    print("")

In [ ]:
#@title ##**Download results** { display-mode: "form" }

files_in_folder = os.listdir(result_folder)
zip_file = "download.zip"

if len(files_in_folder) == 1:
    file_to_download = os.path.join(result_folder, files_in_folder[0])
    files.download(file_to_download)
else:
    if os.path.exists(os.path.join(result_folder, zip_file)):
        os.remove(os.path.join(result_folder, zip_file))
    os.system(f"cd {result_folder} && zip -r -j {zip_file} * && cd ..")
    files.download(os.path.join(result_folder, zip_file))